# Retrieval-Augmented Generation (RAG)

In [1]:
import chromadb
import dotenv
from pathlib import Path
from agents import Agent, Runner, function_tool, trace

dotenv.load_dotenv()

True

## Creamos una vector store de calorías que podemos usar como 'tool':

In [ ]:
# Poblamos la base de datos vectorial con los datos del archivo data/calories.csv en
# Ver notebook rag_setup.ipynb

# Ahora vamos a crear un objeto de tipo chromadb que apunte al directorio donde tenemos los embeddings
chroma_client = chromadb.PersistentClient(path="../chroma")

# Ahora vamos a crear una colección de tipo nutrition_db
calories_db = chroma_client.get_collection(name="nutrition_db")  # 'nutrition_db' es el nombre de la colección que creamos en el notebook 'rag_setup.ipynb'


In [ ]:
# Ahora vamos a hacer una consulta a la base de datos para ver si podemos encontrar información sobre 'bananas'
results = calories_db.query(query_texts=["banana"], 
                             n_results=2)      # n_results es el número de resultados que queremos obtener
                             
for i, doc in enumerate(results["documents"][0]):
    print(sorted(results["metadatas"][0][i].items()))
    print(doc)
    print("\n")

[('calories_per_100g', 89.0), ('food_category', 'fruits'), ('food_item', 'banana'), ('keywords', 'banana_fruits'), ('kj_per_100g', 374.0), ('serving_info', '100g')]
Food: Banana
        Category: Fruits
        Nutritional Information:
        - Calories: 89 per 100g
        - Energy: 374 kJ per 100g
        - Serving size reference: 100g

        This is a fruits food item that provides 89 calories per 100 grams.


[('calories_per_100g', 50.0), ('food_category', '(fruit)juices'), ('food_item', 'banana juice'), ('keywords', 'banana_juice_(fruit)juices'), ('kj_per_100g', 210.0), ('serving_info', '100ml')]
Food: Banana Juice
        Category: (Fruit)Juices
        Nutritional Information:
        - Calories: 50 per 100g
        - Energy: 210 kJ per 100g
        - Serving size reference: 100ml

        This is a (fruit)juices food item that provides 50 calories per 100 grams.




In [ ]:
# Creamos la función que se encargará de buscar la información en la base de datos de embeddings
# Es importante crear un docstring para la función que describa su comportamiento, datos de entrada y salida
@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Function Tool para una base de datos RAG que busca información de calorías para alimentos específicos (pero no para comidas completas.

    Args:
        query: El alimento a buscar.
        max_results: El número máximo de resultados a devolver.

    Returns:
        Una cadena de texto conteniendo la información nutricional.
    """

    results = calories_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Formatear resultados para el agente
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Informacion nutricional:\n" + "\n".join(formatted_results)

Hagamos una prueba de esta función:

> todavia no es una tool... es solo una función

__La siguiente celda solo funcionara con la definicion de funcion. No lo hará si la convertimos en una tool con `@function_tool` __

In [ ]:
#calorie_lookup_tool('bananas')

'Informacion nutricional:\nBanana (Fruits): 89.0 calories per 100g\nBanana Juice ((Fruit)Juices): 50.0 calories per 100g\nBanana Nut Bread (Pastries,Breads&Rolls): 326.0 calories per 100g'

### Creacion del Agente que usa 'tools'

In [ ]:
# Primero convertimos la funcion en una tool con @function_tool
# y luego creamos un agente que use esta tool
# le agregamos la tool a la lista de tools
calorie_agent = Agent(
    name="Asistente nutricional",
    instructions="""
    Eres un asistente nutricional que proporciona información sobre calorías.
    Das respuestas concisas.
    Si necesitas buscar información de calorías, usa el calorie_lookup_tool.
    """,
    tools=[calorie_lookup_tool],
)

In [10]:
with trace("Asistente nutricional con RAG"):
    result = await Runner.run(
        calorie_agent,
        "Cuantas calorias en total tienen una banana y una manzana?" ## Also give calories per 100g",
    )
    print(result.final_output)

Aprox. 200 kcal en total (banana mediana ~105 kcal + manzana mediana ~95 kcal). Cambia con tamaños.
